In [1]:
import numpy as np
import cv2
import tensorflow as tf

In [2]:
# load face detection model (DNN OpenCV)
face_detection_model =cv2.dnn.readNetFromCaffe("./models/deploy.prototxt.txt", 
                                              "./models/res10_300x300_ssd_iter_140000_fp16.caffemodel")

# Load model đã được train từ trước 
model = tf.keras.models.load_model("./model_trained/FaceMaskedTrainedModel02.h5")

2021-12-09 15:12:29.708314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 15:12:29.768803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 15:12:29.768966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 15:12:29.769549: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [10]:
# Test predict
a = np.random.rand(1, 100, 100, 3)
print(model.layers[0].input_shape)
print(a.shape)
print(model.predict(a))

(None, 100, 100, 3)
(1, 100, 100, 3)
[[0.00059103 0.4185522  0.28809175 0.2927651 ]]


In [4]:
# labels
labels = ['Mask', 'No Mask', 'Covered Mouth Chin', 'Covered Nose Mouth']

def get_color(label):
    color = None
    if label == "Mask": 
        color = (0, 255, 0) # Green
    elif label == "No Mask":
        color = (0, 0, 255) # red
    elif label == "Covered Mouth Chin": 
        color = (0, 255, 255)
    else: 
        color = (255, 255, 0)
    return color

In [5]:
get_color("Mask")

(0, 255, 0)

## Predict a image

In [6]:
def face_mask_prediction(img):
    image = img.copy()
    h, w = image.shape[:2]
    # hình ảnh array | scaleFactor | size image | mean value RGB | swapRB
    blob = cv2.dnn.blobFromImage(image, 1, (300, 300), (104, 117, 123), swapRB=True)
    
    # forwading face_detection_model (OpenCV)
    face_detection_model.setInput(blob)
    detection = face_detection_model.forward() 
    for i in range(0, detection.shape[2]):
        confidence = detection[0, 0, i, 2]
        if confidence > 0.5:
            box = detection[0, 0, i, 3:7] * np.array([w, h, w, h])
            box = box.astype(int)
            point1 = (box[0], box[1])
            point2 = (box[2], box[3])

            # Step 2: Data preprocessing
            face = image[box[1]:box[3], box[0]:box[2]]
            face_blob = cv2.dnn.blobFromImage(face, 1, (100, 100), (104, 117, 123), swapRB=True)
            face_blob_squeeze = np.squeeze(face_blob).T
            face_blob_rotate = cv2.rotate(face_blob_squeeze, cv2.ROTATE_90_CLOCKWISE)
            face_blob_flip = cv2.flip(face_blob_rotate, 1)
            
            # Normalization and remove negative value
            max_value = face_blob_flip.max()
            min_value = face_blob_flip.min()
            img_norm = (face_blob_flip - min_value) / (max_value - min_value)
            # img_norm = np.maximum(face_blob_flip, 0) / face_blob_flip.max()
            
            # Step 3: Deep learning (CNN)
            img_input = np.expand_dims(img_norm, 0)
            result = model.predict(img_input)[0]
            confidence_index = result.argmax()
            confidence_score = result[confidence_index]
            label = labels[confidence_index]
            label_text = f'{label}: {(confidence_score*100):.2f}%'

            # color
            color = get_color(label)
            cv2.rectangle(image, point1, point2, color, 4)
            cv2.putText(image, label_text, (point1[0], point1[1]-10), cv2.FONT_HERSHEY_DUPLEX, 1, color, 1)
            
    return image

In [9]:
# Thực hiện test function
img = cv2.imread("./Image_Test/women-wearing-mask.jpg")
image_predict = face_mask_prediction(img)

# cv2.imshow("Original", img)
cv2.imshow("Image predict", image_predict)
cv2.waitKey()
cv2.destroyAllWindows()

## Real time Face Mask Detection

In [12]:
cap = cv2.VideoCapture(0)

while True: 
    ret, frames = cap.read()
    if ret == False:
        break
        
    image = face_mask_prediction(frames)
    cv2.imshow("Face mask prediction", image)
    if cv2.waitKey(1) == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()